In [1]:
%pip install pdf2image python-docx pillow

Note: you may need to restart the kernel to use updated packages.


In [2]:
%pip install poppler-utils

Note: you may need to restart the kernel to use updated packages.


In [3]:

%pip install poppler-utils

Note: you may need to restart the kernel to use updated packages.


In [4]:
%pip install google-cloud-vision

Note: you may need to restart the kernel to use updated packages.


In [5]:
import shutil

poppler_path = shutil.which('pdftoppm')
print(f"Found pdftoppm at: {poppler_path}")

Found pdftoppm at: None


In [ ]:
try:
    from google.colab import files
except ImportError:
    # Fallback for non-Colab environments
    import tkinter as tk
    from tkinter import filedialog
    import sys

    class DummyFiles:
        @staticmethod
        def upload():
            root = tk.Tk()
            root.withdraw()
            file_path = filedialog.askopenfilename(
                title="Select a PDF, DOCX, or TXT file",
                filetypes=[("All Supported", "*.pdf *.docx *.txt"), ("PDF files", "*.pdf"), ("DOCX files", "*.docx"), ("Text files", "*.txt")]
            )
            if not file_path:
                print("No file selected.")
                sys.exit(0)
            filename = os.path.basename(file_path)
            with open(file_path, "rb") as f:
                content = f.read()
            return {filename: content}
    files = DummyFiles()

from pdf2image import convert_from_path
from docx import Document
from PIL import Image, ImageDraw, ImageFont
import os
import textwrap
import uuid
from IPython.display import display, Image as IPImage
import io
import shutil

# Create output directory
OUTPUT_FOLDER = 'output'
os.makedirs(OUTPUT_FOLDER, exist_ok=True)

def allowed_file(filename):
    return '.' in filename and filename.rsplit('.', 1)[1].lower() in {'pdf', 'docx', 'txt'}

def text_to_image(text, output_path, max_width=800, font_size=20):
    lines = textwrap.wrap(text, width=80)
    font = ImageFont.load_default()  # Using default font for simplicity
    line_height = font_size + 5
    image_height = len(lines) * line_height + 40
    image = Image.new('RGB', (max_width, image_height), color='white')
    draw = ImageDraw.Draw(image)

    y_text = 20
    for line in lines:
        draw.text((10, y_text), line, font=font, fill='black')
        y_text += line_height

    image.save(output_path)
    return output_path


# Upload file
print("Please upload a PDF, DOCX, or TXT file:")
uploaded = files.upload()

if not uploaded:
    print("No file uploaded.")
else:
    for filename, content in uploaded.items():
        if not allowed_file(filename):
            print("Invalid file type. Please upload a PDF, DOCX, or TXT file.")
            continue

        # Save uploaded file
        file_path = os.path.join(OUTPUT_FOLDER, filename)
        with open(file_path, 'wb') as f:
            f.write(content)

        image_paths = []
        unique_id = str(uuid.uuid4())
        ext = filename.rsplit('.', 1)[1].lower()

        try:
            if ext == 'pdf':
                # Convert PDF to images
                poppler_path = shutil.which('pdftoppm')
                if poppler_path:
                    images = convert_from_path(file_path, dpi=300, poppler_path=os.path.dirname(poppler_path))
                    for i, image in enumerate(images):
                        output_path = os.path.join(OUTPUT_FOLDER, f"{unique_id}_page_{i+1}.jpg")
                        image.save(output_path, 'JPEG')
                        image_paths.append(output_path)
                else:
                     print("Error: pdftoppm not found. Please ensure poppler-utils is installed and in your system's PATH.")


            elif ext == 'docx':
                doc = Document(file_path)
                text = '\n'.join([para.text for para in doc.paragraphs if para.text])
                output_path = os.path.join(OUTPUT_FOLDER, f"{unique_id}.jpg")
                text_to_image(text, output_path)
                image_paths.append(output_path)

            elif ext == 'txt':
                with open(file_path, 'r', encoding='utf-8') as f:
                    text = f.read()
                output_path = os.path.join(OUTPUT_FOLDER, f"{unique_id}.jpg")
                text_to_image(text, output_path)
                image_paths.append(output_path)

        except Exception as e:
            print(f"Error processing file: {str(e)}")


        # # Display and provide download links for images
        # for image_path in image_paths:
        #     print(f"Generated image: {image_path}")
        #     display(IPImage(filename=image_path))
        #     files.download(image_path)

        # Clean up uploaded file
        os.remove(file_path)


ModuleNotFoundError: No module named 'google.colab'

In [9]:
from google.cloud import vision
import io
import os

# Path to the folder containing the images
image_folder_path = "/content/output"
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "/content/rag-bot-461623-d7d0a008c23f.json"

# Initialize the Vision API client
client = vision.ImageAnnotatorClient()

extracted_text = ""

# Iterate through all files in the image folder
for filename in sorted(os.listdir(image_folder_path)):
    if filename.lower().endswith(('.png', '.jpg', '.jpeg')):
        image_path = os.path.join(image_folder_path, filename)
        print(f"Processing image: {filename}")

        # Read the image file
        with io.open(image_path, 'rb') as image_file:
            content = image_file.read()

        image = vision.Image(content=content)

        # Perform text detection
        response = client.document_text_detection(image=image)

        if response.error.message:
            print(f"API Error processing {filename}: {response.error.message}")
            continue # Skip to the next image if there's an error

        # Append extracted text, adding a separator for clarity between pages
        extracted_text += f"\n===== Page {filename} =====\n"
        extracted_text += response.full_text_annotation.text
        extracted_text += "\n"

# Print the combined extracted text
print("\n===== Combined Extracted Text =====")
print(extracted_text)

# Optional: Save the combined text to a file
with open("extracted_text.txt", "w", encoding="utf-8") as f:
    f.write(extracted_text)
print("\n✅ Combined extracted text saved to 'extracted_text.txt'")

Streaming output truncated to the last 5000 lines.
کی فصل لگائی جاسکتی ہے۔
متان، دباری، خانیوال، بہاولپور علیہ ، ڈی ٹیم جھاڑی نما پودا
ینی خان ، رحیم یارخان
CRI MNH1050 مشان 2300017500
CEMB-100
33
CEMB 16000 18000 پنجاب کے وہ تمام علاقہ جات جہاں پر ٹیم جھاڑی نما
کپاس کی فصل لگائی جا سکتی ہے
34 WE AL وین گروپ 12000 14000 پنجاب کے وہ تمام علاقہ جات جہاں پر جھاڑی دار
35
AG-9
کپاس کی فصل لگائی جا سکتی ہے۔ وہ رقبے
جہاں پانی اور دوسرے وسائل کم ہوں بہتر
پیداوار دیتی ہے
WEAL المداد مین گروپ 22000 25000 پنجاب کے وہ تمام علاقہ جات جہاں پر در میانه قد
AG-1606
CIM-
36
2022
343
سمیاس کی فصل لگائی جاسکتی ہے
CCRIتان 17500 23000 متان: لودھراں، بہاولپور: وہاڑی، خانیوال
زی جی خان، رحیم یا ر خان، بہاولنگر، راجن نیم جھاڑی نما در میانہ قد
یاد مظفر گڑھ
50:25
55:50

===== Page 3d24fffd-b1ef-4f44-941f-34f09e7bfa73_page_14.jpg =====
65:60
65060
6045
55-45
55-45
45-40
6055
55:50
55-50
14
37 537-CCRI CYTOستان 17500 23000 سلمان، لودھراں، بہاولپور، وہاڑی، خانیوال،
38
39
2022
775-CCRI
CIM
ڈی جی خان، رحیم یا ر خان، 